In [13]:
import json
import re
import glob
from PIL import Image as PImage
from transformers import DonutProcessor
from transformers import VisionEncoderDecoderModel
import torch
import os


pro_dir = "last_trained_donut_pro"
mod_dir = "last_trained_donut_mod"
con_dir = "last_trained_donut_con"

os.environ['PYTORCH_HIP_ALLOC_CONF']='expandable_segments:True'
path = "Input/W2_Single_Clean_jpg/"
# set this variable as true if you want to perform your own sanity checks to make sure the data is correct
sanity_check = False

In [14]:

def load_images(path):
    files = glob.glob(f"{path}/*.jpg")
    loaded_images = []
    for f in files:
        img = PImage.open(f)
        loaded_images.append(img)
    return loaded_images

imgs = load_images(path)
if sanity_check:
    imgs[0].show()


In [15]:
# setting device on GPU if available, else CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
#device = torch.device('cpu')
print('Using device:', device)
print()

Using device: cuda



In [ ]:
image = imgs[2]

#After searching a bit, Donut seems to be the best OCR replacement. This version the finetuned-docvqa is ideal for more generalized 'find thing in pdf' usage. 
processor = DonutProcessor.from_pretrained("naver-clova-ix/donut-base")

model = VisionEncoderDecoderModel.from_pretrained(mod_dir)

model.to(device)
def question_result(model, question:str, image):
    task_prompt = f"<s_docvqa><s_question>{question}</s_question><s_answer>"

    decoder_input_ids = processor.tokenizer(task_prompt, add_special_tokens=False, return_tensors="pt").input_ids

    pixel_values = processor(image, return_tensors="pt").pixel_values

    outputs = model.generate(

        pixel_values.to(device),

        decoder_input_ids=decoder_input_ids.to(device),

        max_length=model.decoder.config.max_position_embeddings,
 
        pad_token_id=processor.tokenizer.pad_token_id,

        eos_token_id=processor.tokenizer.eos_token_id,

        use_cache=True,

        bad_words_ids=[[processor.tokenizer.unk_token_id]],

        return_dict_in_generate=True,

    )
    end_token = "</s_question><s_answer>"
    start_token = "<s_docvqa><s_question>"
    string_fixer = len(start_token) + len(end_token) + len(question)
    sequence = processor.batch_decode(outputs.sequences)[0]

    sequence = sequence.replace(processor.tokenizer.eos_token, "").replace(processor.tokenizer.pad_token, "")
    good_enough = sequence[string_fixer:]
    sequence = re.sub(r"<.*?>", "", sequence, count=1).strip()  # remove first task start token
    result = processor.token2json(sequence)
    result = good_enough
    return result



Config of the encoder: <class 'transformers.models.donut.modeling_donut_swin.DonutSwinModel'> is overwritten by shared encoder config: DonutSwinConfig {
  "attention_probs_dropout_prob": 0.0,
  "depths": [
    2,
    2,
    14,
    2
  ],
  "drop_path_rate": 0.1,
  "embed_dim": 128,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.0,
  "hidden_size": 1024,
  "image_size": [
    1280,
    960
  ],
  "initializer_range": 0.02,
  "layer_norm_eps": 1e-05,
  "mlp_ratio": 4.0,
  "model_type": "donut-swin",
  "num_channels": 3,
  "num_heads": [
    4,
    8,
    16,
    32
  ],
  "num_layers": 4,
  "patch_size": 4,
  "path_norm": true,
  "qkv_bias": true,
  "torch_dtype": "float32",
  "transformers_version": "4.49.0",
  "use_absolute_embeddings": false,
  "window_size": 10
}

Config of the decoder: <class 'transformers.models.mbart.modeling_mbart.MBartForCausalLM'> is overwritten by shared decoder config: MBartConfig {
  "activation_dropout": 0.0,
  "activation_function": "gelu",
  "add_cro

In [17]:
Questions_o_Concern = ['What is the social security number?', 
                       "What is the Employer identification number?", 
                       "What is the Employee's first and last Name?", 
                       "What is the Employer's name?", 
                       "What is the Employee's zip code?",
                       "What is the wages and tips?", 
                       "what is the address that begins with 365?", 
                       "What is the year of Wage and Tax Statement?", 
                       "What is the information being furnished to?", 
                       "Ignore watermarks, what is the employee's address?", 
                       "What is the zipcode?", "What's inside of box e?", 
                       "What is the state inside of box e?", 
                       "What is the employee's Address?", 
                       "What is the data inside of box f?", 
                       "What is the Employer's state ID number?", 
                       "What is the second Employer's state ID number?"]

In [ ]:
for q in Questions_o_Concern:
    print(f"{q} : {question_result(model, q, image)}")

TypeError: 'JpegImageFile' object is not subscriptable